In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import os
from scipy.optimize import curve_fit
from bokeh.plotting import figure, show, output_notebook#, output_file
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.models.widgets import Panel, Tabs, Select
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox
from get_lactate_from_excel import get_lactate_from_excel
from pace_convertion import pace_convertion

sport_type = 'bike' # 'swim', 'bike', 'run'
file_path = 'test_dataframe.xlsx'

# 从get_lactate_from_excel.py中获取计算结果的列表
if os.path.exists(file_path):
    lactate = get_lactate_from_excel(file_path, sport_type)

    ###### 以下为各个变量 #######
    ae_intensity_list = [pace_convertion(i, sport_type) for i in lactate[1]] # 提取有氧测试强度数值列表
    ae_lactate_list = lactate[0] # 提取有氧测试乳酸值列表
    peak_lact_min = lactate[4] # 提取无氧乳酸峰值出现的分钟数
    peak_lactate = lactate[3] # 提取无氧乳酸峰值
    an_peak_intensity = pace_convertion(lactate[2], sport_type) # 提取无氧测试强度
    recovery_lact = lactate[5] # 提取无氧测试恢复20分钟后的乳酸值
    recovery_rate = str(round((peak_lactate - recovery_lact) / peak_lactate * 100, 2)) + '%' # 乳酸清楚速率
    ref_lac_value = 4 # 设置乳酸参考点（数值较大的那个点，另一个点自动取该值减2）
    
    ##### Debug ####
    #print(lactate[1])
    #print(ae_intensity_list)
    #print(ae_lactate_list)
    

    if sport_type == 'bike': # 根据运动确定X轴单位
        x_unit = ['Power Output', 'Watt']
    else:
        x_unit = ['Speed', 'm/s']

    # 格式化有氧测试数据为np float64
    x = np.array(ae_intensity_list).astype(np.float64)
    y = np.array(ae_lactate_list).astype(np.float64)

    # 建立指数公式
    def func(x, a, b):
        return a*(b**x)

    # 获取指数回归公式的参数a和b
    popt, pcov = curve_fit(func, x, y)
    a = popt[0] # 得到系数a
    b = popt[1] # 得到系数b

    # 获取拟合指数曲线
    x2 = np.arange(x[0], x[-1], 0.1) # 取有氧测试最小和最大强度的range列表作为拟合曲线的X值
    y2 = func(x2, a, b) # 算出拟合后的Y值

    lac_value_list = [ref_lac_value-2, ref_lac_value] # 生成要取的乳酸值参考点列表，两个点差值为2mM

    # 获取指定血乳酸值对应的强度
    def get_intensity(ref_lac_value, a, b): # 获得2个乳酸参考点对应的强度值
        return [round((np.log((ref_lac_value-2)/a))/(np.log(b)), 2), round((np.log(ref_lac_value/a))/(np.log(b)), 2)]

    # 计算拟合优度 R2值
    # residual sum of squares
    ss_res = np.sum((y - func(x, a, b)) ** 2)
    # total sum of squares
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    # r-squared
    r2 = 1 - (ss_res / ss_tot)
    #print('拟合优度: ' + str(round(r2, 3)))

    '''
    ###### 下面使用Matplotlib作图，有氧部分 #######
    plt.figure()
    plt.axis([int(float(ae_intensity_list[0])*0.7), int(float(ae_intensity_list[-1])*1.2), 0, int(ae_lactate_list[-1]*1.3)]) # 建立x和y坐标
    plt.plot(x, y, '^--', label="Original") # 原始数据
    plt.plot(x2, y2, 'r-', label="Fitted  (R=" + str(round(r2, 3)) + ")") # 拟合曲线
    plt.plot(get_intensity(ref_lac_value, a, b), lac_value_list, 'go', label=str(lac_value_list[0])+'mM & '+str(lac_value_list[1])+'mM Lactate')
    plt.xlabel('Power Output')
    plt.ylabel('Lactate (mM)')
    plt.legend(loc=2)
    plt.show()
    '''

    ###### 下面使用 Bokeh 作图 ######

    # 输出静态到HTML文件
    #output_file("lactate_lines.html")

    # 输出到 Jupyter Notebook
    output_notebook()

    # 创建一个新的图表
    p = figure(
        tools="pan,box_zoom,reset,save", # 显示工具栏
        x_range=[int(float(ae_intensity_list[0])*0.7), int(float(ae_intensity_list[-1])*1.2)], # X轴坐标
        y_range=[0, int(ae_lactate_list[-1]*1.3)], # Y轴坐标
        title="Aerobic Testing - " + sport_type.title(), # 图表标题
        x_axis_label=x_unit[0]+" ("+x_unit[-1]+")", y_axis_label='Lactate (mM)' # X和Y轴的标签
        )

    source = ColumnDataSource(data=dict(intensity=get_intensity(ref_lac_value, a, b), # 给两个乳酸参考点建立参考标签
                                        lactate_value=lac_value_list,
                                        markers=[str(round(get_intensity(ref_lac_value, a, b)[0], 2))+' '+x_unit[-1],
                                                 str(round(get_intensity(ref_lac_value, a, b)[-1], 2))+' '+x_unit[-1]]))
    labels = LabelSet(x='intensity', y='lactate_value', text='markers', level='glyph',
                  x_offset=-85, y_offset=10, source=source, render_mode='canvas',
                  border_line_color='black', border_line_alpha=1.0,
                  background_fill_color='white', background_fill_alpha=1.0)

    # 在图上添加点和线
    p.triangle(x, y, legend="Original", fill_color="white", line_color="blue", line_width=2, size=12) # 表示原始数据的点
    p.line(x, y, legend="Original", line_color="blue", line_dash="4 4", line_width=2) # 原始数据的虚线连线
    p.line(x2, y2, legend="Fitted  (R2=" + str(round(r2, 3)) + ")", line_color="red", line_width=3) # 拟合指数曲线
    p.circle(get_intensity(ref_lac_value, a, b), lac_value_list,
             legend=str(lac_value_list[0])+'mM & '+str(lac_value_list[1])+'mM Lactate',
             fill_color="green", line_color="green", size=12, alpha=0.7) # 2个乳酸参考点
    p.quad(top=[ref_lac_value, ref_lac_value-2], # 画出2mM和4mM方形区域
           bottom=[ref_lac_value-2, 0],
           left=[int(float(ae_intensity_list[0])*0.7), int(float(ae_intensity_list[0])*0.7)],
           right=[int(float(ae_intensity_list[-1])*1.2), int(float(ae_intensity_list[-1])*1.2)],
           line_color="grey", line_dash="4 4", line_width=3, color=["grey", "grey"], alpha=[0.2,0.1])
    p.legend.location = "top_left"
    p.title.align = 'center'
    p.title.text_font_size = "14px"
    p.add_layout(labels) # 建立标签
    tab1 = Panel(child=p, title="Aerobic Testing - " + sport_type.title())

    # 显示数据图
    #show(p)


    # 创建一个新的图表
    p2 = figure(
        tools="pan,box_zoom,reset,save", # 显示工具栏
        x_range=[0, 30], # X轴坐标
        y_range=[0, int(peak_lactate*1.3)], # Y轴坐标
        title="Anaerobic & Recovery Testing - " + sport_type.title(), # 图表标题
        x_axis_label='Minutes (after 25s all-out effort)', y_axis_label='Lactate (mM)' # X和Y轴的标签
        )

    source2 = ColumnDataSource(data=dict(time2=[peak_lact_min, 20], # 给两个乳酸参考点建立标签
                                        lactate2=[peak_lactate, recovery_lact],
                                        markers2=[str(peak_lactate)+" mM",
                                                 str(recovery_lact)+" mM"]))
    labels2 = LabelSet(x='time2', y='lactate2', text='markers2', level='glyph',
                  x_offset=10, y_offset=8, source=source2, render_mode='canvas',
                  border_line_color='black', border_line_alpha=1.0,
                  background_fill_color='white', background_fill_alpha=1.0)

    # 在图上添加点和线
    p2.triangle(peak_lact_min, peak_lactate, legend="Peak Lactate (@ " + str(round(an_peak_intensity, 2)) + x_unit[-1] + ")",
                fill_color="orange", line_color="red", line_width=2, size=12) # 表示无氧乳酸峰值
    p2.circle(20, recovery_lact, legend="Recovered in 20mins (Rate="+recovery_rate+")", line_width=2, size=12 ) # 20分钟后的乳酸值
    p2.line([peak_lact_min, 20], [peak_lactate, recovery_lact], line_color="blue", line_dash="4 4", line_width=2)  # 连接两个点
    p2.legend.location = "top_right" 
    p2.title.align = 'center'
    p2.title.text_font_size = "14px"
    p2.add_layout(labels2) # 建立标签
    tab2 = Panel(child=p2, title="Anaerobic & Recovery Testing - " + sport_type.title())

    # 显示数据图
    #show(p2)

    tabs = Tabs(tabs=[ tab1, tab2 ])

    show(tabs)
else:
    print("WARNING: Error! File '" + file_path + "' Does Not Exist!")

Loading BokehJS ...